In [1]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
import glob
from tqdm.auto import tqdm
import country_converter as coco
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

2025-10-22 13:25:00.283 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
df = pl.read_csv(f'{RAW_DATA_DIR}/zmap_log_2025-10-21T09_18_27+00_00.log.zst')

In [7]:
df.head()

initial-ip,saddr,ttl,sent_timestamp_ts,sent_timestamp_us,timestamp_ts,timestamp_us,classification,type,code,icmp-id
str,str,i64,i64,i64,i64,i64,str,i64,i64,i64
"""2000:b70:25::""","""2a01:6420:8030:1::1""",63,96,1010446336,1761038310,536088,"""unreach_noroute""",1,0,0
"""2001:254:86a3::""","""2a01:6420:8030:1::1""",63,96,1010446336,1761038310,547763,"""unreach_noroute""",1,0,0
"""2001:254:2a9d::""","""2a01:6420:8030:1::1""",63,96,1010446336,1761038310,547773,"""unreach_noroute""",1,0,0
"""2001:254:518e::""","""2a01:6420:8030:1::1""",63,96,1010446336,1761038310,548108,"""unreach_noroute""",1,0,0
"""2001:468:7358::""","""2a01:6420:8030:1::1""",63,96,1010446336,1761038310,549072,"""unreach_noroute""",1,0,0


In [6]:
df.filter(pl.col('classification')=='timxceed').select(pl.col('initial-ip')).n_unique()

146435076

In [9]:
df_ampl = df.filter(pl.col('classification')=='timxceed').group_by('initial-ip').agg(
    pl.len().alias('amplification')
)

In [10]:
df_ampl = df_ampl.filter(pl.col('amplification')>1)

In [11]:
df_ampl

initial-ip,amplification
str,u32
"""2a06:a402:459c::""",2
"""2a01:75c1:7161::""",2
"""2804:1498:7028::""",2
"""2804:3624:4f96::""",2
"""2804:ce4:35bc::""",2
…,…
"""2804:585c:667d::""",3
"""2804:394:8859::""",2
"""2804:30b0:4c00::""",2
